## Read fitness data from csv

In [ ]:
import pandas as pd
fitness_data = pd.read_csv("./data/health-data.csv")

In [ ]:
fitness_data.head()

## Split real & mock data

In [ ]:
real_data = fitness_data[0:7]
real_data

In [ ]:
mock_data = fitness_data[7:]
mock_data

## Initiate connection to Syft server

#### import pysyft

In [ ]:
try:
    import syft as sy
    print("Syft is imported")
except:
    print("Syft is not installed")

#### open connection to server
- hostname = localhost
- port = 8080
- username = info@openmined.org
- password = changethis

In [ ]:
try:
    domain_client = sy.login(
        port=8080,
        email="info@openmined.org",
        password="changethis"
    )
except Exception as e:
    print("Unable to login")

## Create Data Subject

In [ ]:
data_subject = domain_client.data_subject_registry.get_all()
data_subject

In [ ]:
fitness_data_subject = sy.DataSubject(name="Fitness", aliases=['fitness'])
fitness_A = sy.DataSubject(name="Fitness A", aliases=['fitness:a'])

In [ ]:
fitness_data_subject.add_member(fitness_A)

In [ ]:
fitness_data_subject.members

In [ ]:
# upload data subject into server
domain_client.data_subject_registry.add_data_subject(fitness_data_subject)

In [ ]:
data_subject = domain_client.data_subject_registry.get_all()
data_subject

## Create Dataset

In [ ]:
dataset = sy.Dataset(name="Fitness A user data")
dataset.set_description("10 users health data")

## Create Asset

In [ ]:
ctf = sy.Asset(name="fitness example asset")
ctf.set_description("10 example data of users in fitness A")

In [ ]:
# set real data
ctf.set_obj(real_data)
ctf.set_shape(real_data.shape)

# set mock data
ctf.set_mock(mock_data, mock_is_real=False)

In [ ]:
# attach data subject to asset
ctf.add_data_subject(fitness_A)

In [ ]:
dataset.add_asset(ctf)

## Upload dataset to server

In [ ]:
domain_client.upload_dataset(dataset)

#### Verify uploaded dataset

In [ ]:
dataset_on_server = domain_client.datasets.get_all()
dataset_on_server

#### Access data
- real & mock data should be accessible

In [ ]:
# access mock dataset
mock = domain_client.datasets[0].assets[0].mock
mock

In [ ]:
# access real dataset
real = domain_client.datasets[0].assets[0].data
real

## Create data scientist account

In [ ]:
domain_client.register(
    name="Data Scientist 1",
    email="datasci@edu.edu",
    password="abc123",
    password_verify="abc123",
    institution="Data Scientist account"
)

## Approve code from data scientist

In [ ]:
project = domain_client.projects[0]

In [ ]:
project

#### see code in request

In [ ]:
request = project.requests[0]
request

In [ ]:
request.code

#### see involing asset

In [ ]:
request.code.assets[0].mock

In [ ]:
request.code.assets[0].data

#### Try to run code

In [ ]:
# get user function
scientist_function = request.code.unsafe_function

In [ ]:
real_result = scientist_function(fitness_data=request.code.assets[0].data)
real_result

## Share result back to data scientist

In [ ]:
result = request.accept_by_depositing_result(real_result)

In [ ]:
result